# CW1 - Multimodal IMDB Analysis with Keras

## Introduction

In this assignment you will be provided a dataset containing a selection of films together with their posters in JPEG image format and their overviews in text format from the Internet Movie Database.

![Example](https://i.ibb.co/J3trT62/Screenshot-2024-09-22-214753.png)

You will be analysing this dataset by implementing and training two models: a **CNN** and an **LSTM**.

The CNN must classify film posters by the genre. Independently, the LSTM must classify film overviews by the genre. Finally, you will evaluate and critically comment your results in a short report. (Which of the two models was better at classifying films?)

## Structure of the assignment

This assignment is broken up into sections and you need to complete each section successively. The sections are the following:

1. Data Processing

  1.a. Image processing of the posters

  1.b. Natural language processing of the overviews

2. Definition of the models

  2.a. CNN for the posters

  2.b. LSTM for the overviews

3. Training of the models
4. Evaluation of the models

In addition to this coding exercise, you must write a **2-3 pages** report analysing and critically evaluating your model's results. Marks for the report will be awarded for depth of analysis and critical thinking skills. You should consider how well your model performs and WHY it does that—give specific examples and comment on their importance.

In [2]:
# Enter your module imports here, some modules are already provided

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import pathlib
import pandas as pd
from sklearn import model_selection
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.metrics import Precision, Recall
from tensorflow.keras.layers import Input, Conv2D, Dropout, MaxPooling2D, Flatten, Dense

In [3]:
# CodeGrade Tag Init1
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Data Processing

Warning: running the following cell can take some time.


In [4]:
# CodeGrade Tag Init2
# Do not edit, remove, or copy this cell

# This code will copy the images from your google drive into the colab file
# storage. Make sure the dataset is unzipped in your drive folder.
! mkdir /tmp/Multimodal_IMDB_dataset
! rsync -ah --info=progress2 /content/drive/MyDrive/Multimodal_IMDB_dataset/Images /tmp/Multimodal_IMDB_dataset/


mkdir: cannot create directory ‘/tmp/Multimodal_IMDB_dataset’: File exists
        192.50M 100%    1.09MB/s    0:02:48 (xfr#7931, to-chk=0/7932)


In [5]:
# CodeGrade Tag Init3
# Do not edit, remove, or copy this cell

# Load the csv with the sample IDs and genres
csv_loc = "/content/drive/MyDrive/Multimodal_IMDB_dataset/IMDB_overview_genres.csv"
dataset = pd.read_csv(csv_loc)

# Split the dataset into training and testing
data_train, data_test = model_selection.train_test_split(dataset, test_size=0.2)

# Convert the labels to arrays
labels_train = np.array(data_train.drop(['Id', 'Genre', 'Overview'],axis=1)).astype('float32')
labels_test = np.array(data_test.drop(['Id', 'Genre', 'Overview'],axis=1)).astype('float32')

# List of the genre names
genres = np.array(data_train.drop(['Id', 'Genre', 'Overview'],axis=1).columns)

# List of overviews
overviews_train = np.array(data_train['Overview'])
overviews_test = np.array(data_test['Overview'])

# Build the file locations for the images
img_loc = "/tmp/Multimodal_IMDB_dataset/Images"

img_locs_train = np.array([[img_loc + '/' + id + '.jpg' for id in data_train['Id']]])
img_locs_test = [[img_loc + '/' + id + '.jpg' for id in data_test['Id']]]

# This function is provided to read in the image files from the folder
def parse_image(filename, label):
  image = tf.io.read_file(filename)
  image = tf.io.decode_jpeg(image)
  return image, label


### 1.a. Image processing of the posters

In [6]:
# CodeGrade Tag Init4
# Do not edit, remove, or copy this cell

# Create the initial datasets of film posters
list_posters_train_ds = tf.data.Dataset.from_tensor_slices((img_locs_train[0], labels_train))
list_posters_valid_ds = tf.data.Dataset.from_tensor_slices((img_locs_test[0], labels_test))

* Create a function called ```img_process``` that converts the images to float32 datatype and resizes them to 64x64 pixels

In [7]:
# CodeGrade Tag Ex1a-i
### Create a function called img_process that converts the images to
### float32 datatype and resizes them to 64x64 pixels

def img_process(image, label):
    # Complete here
    # Read the image file
    img = tf.io.read_file(image)
    # Decode the image
    img = tf.image.decode_jpeg(img, channels=3)
    # Resize the image to 64x64 pixels
    img = tf.image.resize(img, [64, 64])
    # Convert the image to float32
    img = tf.cast(img, tf.float32)
    return img, label


* **Using the ``tf.data`` API, load in the training and validation data for the posters. Be mindful of efficient data processing good practice to minimise the time it takes to load the data.**

In [8]:
# CodeGrade Tag Ex1a-ii
### Use the parse_image and img_process functions to construct the training and
### validation datasets. You should utilise good practice in optimising the
### dataset loading. Use a batch size of 64.

# Define batch size
BATCH_SIZE = 64

# Apply the img_process function and optimize data loading
posters_train_ds = (list_posters_train_ds
                    .map(img_process, num_parallel_calls=tf.data.AUTOTUNE)  # Apply img_process function
                    .batch(BATCH_SIZE)                                     # Batch the data
                    .prefetch(tf.data.AUTOTUNE))                           # Prefetch for efficient loading

posters_valid_ds = (list_posters_valid_ds
                    .map(img_process, num_parallel_calls=tf.data.AUTOTUNE)  # Apply img_process function
                    .batch(BATCH_SIZE)                                     # Batch the data
                    .prefetch(tf.data.AUTOTUNE))                           # Prefetch for efficient loading


### 1.b. Natural Language processing of the overviews

In [9]:
# CodeGrade Tag Init5
# Do not edit, remove, or copy this cell

# Create the initial datasets of the film overviews
list_overviews_train_ds = tf.data.Dataset.from_tensor_slices((overviews_train, labels_train))
list_overviews_valid_ds = tf.data.Dataset.from_tensor_slices((overviews_test, labels_test))

* **Using the ``tf.data`` API, load in the training and validation data for the overviews.**

In [11]:
# CodeGrade Tag Ex1b-i
### Construct the training and validation datasets. Use a batch size of 64.

overviews_train_ds = list_overviews_train_ds.batch(64)

overviews_valid_ds = list_overviews_valid_ds.batch(64)

* Build the vocabulary of the model by calling the ``encoder.adapt()`` method on the film overviews train data.

In [12]:
# CodeGrade Tag Ex1b-ii
### Build the vocabulary of the model by calling the encoder.adapt() method on
### the film overviews train data.

VOCAB_SIZE = 10000  # Define maximum vocabulary size

# Define the TextVectorization layer
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,  # Maximum size of the vocabulary
    output_sequence_length=100  # Sequence length to truncate or pad the sequences
)

# Adapt the encoder to the film overviews training data
encoder.adapt(overviews_train)


* Print the first 200 words of the vocabulary you obtained.

In [13]:
# CodeGrade Tag Ex1b-iii
### Print the first 200 words of the vocabulary you obtained.

# Retrieve the vocabulary from the encoder
vocabulary = encoder.get_vocabulary()

# Print the first 200 words
print(vocabulary[:200])


['', '[UNK]', 'a', 'the', 'to', 'of', 'and', 'in', 'his', 'is', 'an', 'with', 'her', 'for', 'on', 'he', 'their', 'who', 'by', 'from', 'when', 'as', 'that', 'after', 'young', 'they', 'life', 'man', 'two', 'him', 'at', 'new', 'are', 'but', 'into', 'has', 'up', 'she', 'woman', 'one', 'love', 'out', 'family', 'find', 'must', 'it', 'be', 'world', 'friends', 'finds', 'school', 'story', 'them', 'about', 'while', 'where', 'girl', 'have', 'lives', 'group', 'years', 'father', 'home', 'wife', 'help', 'town', 'city', 'all', 'war', 'get', 'three', 'during', 'becomes', 'boy', 'himself', 'back', 'which', 'son', 'york', 'gets', 'high', 'between', 'team', 'against', 'murder', 'time', 'only', 'former', 'american', 'falls', 'daughter', 'takes', 'tries', 'own', 'its', 'other', 'mother', 'become', 'police', 'down', 'order', 'this', 'can', 'will', 'being', 'save', 'was', 'old', 'friend', 'death', 'college', 'small', 'goes', 'before', 'together', 'set', 'over', 'through', 'way', 'agent', 'more', 'take', 'not

# 2. Definition of the models

### 2.a. CNN

**Using the Keras Functional API, create a convolutional neural network with the architecture shown in the model summary below.**

**A few important points to consider:**

* Call the convolutional layers and the first dense layer should have ReLU activation functions. The output layer should have a Sigmoid activation function.
* Pay attention to the output shapes and the number of partmeters for each layer, as these give indications as to the correct settings for the number of filters, kernel size, stride length and padding.
* Use the layer names provided in the summary in your model.
* For the dropout layers, use a dropout rate of 0.2 after the convolutional layers and 0.5 after the dense layers.


```
# Model Summary

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 Input (InputLayer)          [(None, 64, 64, 3)]       0         
                                                                 
 Conv0 (Conv2D)              (None, 32, 32, 16)        448       
                                                                 
 Drop1 (Dropout)             (None, 32, 32, 16)        0         
                                                                 
 Conv1 (Conv2D)              (None, 32, 32, 32)        4640      
                                                                 
 Conv2 (Conv2D)              (None, 32, 32, 32)        9248      
                                                                 
 Drop2 (Dropout)             (None, 32, 32, 32)        0         
                                                                 
 Pool1 (MaxPooling2D)        (None, 16, 16, 32)        0         
                                                                 
 Conv3 (Conv2D)              (None, 16, 16, 64)        18496     
                                                                 
 Conv4 (Conv2D)              (None, 16, 16, 64)        36928     
                                                                 
 Drop3 (Dropout)             (None, 16, 16, 64)        0         
                                                                 
 Pool2 (MaxPooling2D)        (None, 8, 8, 64)          0         
                                                                 
 Conv5 (Conv2D)              (None, 8, 8, 128)         73856     
                                                                 
 Conv6 (Conv2D)              (None, 8, 8, 128)         147584    
                                                                 
 Drop4 (Dropout)             (None, 8, 8, 128)         0         
                                                                 
 Pool3 (MaxPooling2D)        (None, 4, 4, 128)         0         
                                                                 
 Flat (Flatten)              (None, 2048)              0         
                                                                 
 FC1 (Dense)                 (None, 1024)              2098176   
                                                                 
 Drop5 (Dropout)             (None, 1024)              0         
                                                                 
 FC2 (Dense)                 (None, 1024)              1049600   
                                                                 
 Drop6 (Dropout)             (None, 1024)              0         
                                                                 
 Output (Dense)              (None, 25)                25625     
                                                                 
=================================================================
Total params: 3464601 (13.22 MB)
Trainable params: 3464601 (13.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


```



In [14]:
# CodeGrade Tag Ex2a-i
### Define the model using the Keras Functional API. Use the summary above as a
### guide for the model parameters. You will need to define the filters/units of
### the layers correctly, as well as the kernel size, stride length and padding
### of the convolutional layers.

from tensorflow.keras import layers, models

def build_cnn_model():
    # Input layer
    inputs = layers.Input(shape=(64, 64, 3), name="Input")

    # Convolutional layers
    x = layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu', name="Conv0")(inputs)
    x = layers.Dropout(0.2, name="Drop1")(x)

    x = layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', name="Conv1")(x)
    x = layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', name="Conv2")(x)
    x = layers.Dropout(0.2, name="Drop2")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="Pool1")(x)

    x = layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', name="Conv3")(x)
    x = layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', name="Conv4")(x)
    x = layers.Dropout(0.2, name="Drop3")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="Pool2")(x)

    x = layers.Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', name="Conv5")(x)
    x = layers.Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', name="Conv6")(x)
    x = layers.Dropout(0.2, name="Drop4")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="Pool3")(x)

    # Flatten layer
    x = layers.Flatten(name="Flat")(x)

    # Fully connected layers
    x = layers.Dense(units=1024, activation='relu', name="FC1")(x)
    x = layers.Dropout(0.5, name="Drop5")(x)
    x = layers.Dense(units=1024, activation='relu', name="FC2")(x)
    x = layers.Dropout(0.5, name="Drop6")(x)

    # Output layer
    outputs = layers.Dense(units=25, activation='sigmoid', name="Output")(x)

    # Define the model
    model = models.Model(inputs=inputs, outputs=outputs, name="CNN_Model")

    return model

# Build the CNN model
cnn_model = build_cnn_model()

# Print the model summary
cnn_model.summary()



Model: "CNN_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)                   │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv0 (Conv2D)                       │ (None, 32, 32, 16)          │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Drop1 (Dropout)                      │ (None, 32, 32, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv1 (Conv2D)                       │ (None, 32, 32, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv2 (Conv2D)                       │ (None, 32, 32, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Drop2 (Dropout)                      │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Pool1 (MaxPooling2D)                 │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv3 (Conv2D)                       │ (None, 16, 16, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv4 (Conv2D)                       │ (None, 16, 16, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Drop3 (Dropout)                      │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Pool2 (MaxPooling2D)                 │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv5 (Conv2D)                       │ (None, 8, 8, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv6 (Conv2D)                       │ (None, 8, 8, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Drop4 (Dropout)                      │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Pool3 (MaxPooling2D)                 │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Flat (Flatten)                       │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ FC1 (Dense)                          │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Drop5 (Dropout)                      │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ FC2 (Dense)                          │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Drop6 (Dropout)                      │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Output (Dense)                       │ (None, 25)                  │          25,625 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 3,464,601 (13.22 MB)

 Trainable params: 3,464,601 (13.22 MB)

 Non-trainable params: 0 (0.00 B)

* Print the model summary and confirm it has the same architecture as the one provided.

In [15]:
# CodeGrade Tag Ex2a-ii
### Print the model summary and confirm it has the same architecture as the one
### provided.

# Print the model summary
cnn_model.summary()


Model: "CNN_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)                   │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv0 (Conv2D)                       │ (None, 32, 32, 16)          │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Drop1 (Dropout)                      │ (None, 32, 32, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv1 (Conv2D)                       │ (None, 32, 32, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv2 (Conv2D)                       │ (None, 32, 32, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Drop2 (Dropout)                      │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Pool1 (MaxPooling2D)                 │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv3 (Conv2D)                       │ (None, 16, 16, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv4 (Conv2D)                       │ (None, 16, 16, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Drop3 (Dropout)                      │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Pool2 (MaxPooling2D)                 │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv5 (Conv2D)                       │ (None, 8, 8, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv6 (Conv2D)                       │ (None, 8, 8, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Drop4 (Dropout)                      │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Pool3 (MaxPooling2D)                 │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Flat (Flatten)                       │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ FC1 (Dense)                          │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Drop5 (Dropout)                      │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ FC2 (Dense)                          │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Drop6 (Dropout)                      │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Output (Dense)                       │ (None, 25)                  │          25,625 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 3,464,601 (13.22 MB)

 Trainable params: 3,464,601 (13.22 MB)

 Non-trainable params: 0 (0.00 B)

* **Compile the model using the Adam Optimizer with a learning rate of ```1e-4``` and ```binary crossentropy``` loss function. For the metrics, use the ``Precision`` and ``Recall`` functions.**

In [16]:
# CodeGrade Tag Ex2a-iii
### Compile the model using the Adam Optimizer with a learning rate of 1e-4 and
### binary crossentropy loss function. For the metrics, use the Precision and
### Recall functions.

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

cnn_model.compile(
    optimizer=Adam(learning_rate=1e-4),  # Adam optimizer with specified learning rate
    loss='binary_crossentropy',         # Binary crossentropy loss function
    metrics=[Precision(), Recall()]     # Precision and Recall as metrics
)


### 2.b. LSTM model

* Set up the embedding layer by using ```tf.keras.layers.Embedding```. The ```input_dim``` is the length of the vocab, and the ```output_dim``` must be **265**. You should also set ```mask_zero=True```.

In [17]:
# CodeGrade Tag Ex2b-i
### Set up the embedding layer. The input_dim is the length of the vocab, and
### the output_dim must be 256. You should also set mask_zero=True.

embedder = tf.keras.layers.Embedding(
    input_dim=VOCAB_SIZE,  # Length of the vocabulary
    output_dim=256,        # Dimensionality of the embedding space
    mask_zero=True         # Masks padding tokens with 0
)


* Use ```tf.keras.Sequential``` to build a keras sequential model, with the following layers:



  1.   encoder
  2.   embedder
  3.   biLSTM layer with 256 units, dropout 0.5, recurrent dropout 0.2 (make sure to use the right ```return_sequences``` parametre to be able to stack this layer with the following BiLSTM)
  4.   biLSTM layer with 128 units, dropout 0.5, recurrent dropout 0.2
  5.   dense layer with 128 units and relu activation function
  6.   dropout with rate 0.8
  7.   dense output layer with 25 units and sigmoid activation function



In [19]:
# CodeGrade Tag Ex2b-ii
### Build a keras sequential model, with the layers provided above.

lstm_model = tf.keras.Sequential([
    encoder,
    embedder,
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(
            256, dropout=0.5, recurrent_dropout=0.2, return_sequences=True
        ),
        name="BiLSTM_256"
    ),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(
            128, dropout=0.5, recurrent_dropout=0.2, return_sequences=False
        ),
        name="BiLSTM_128"
    ),
    tf.keras.layers.Dense(128, activation="relu", name="Dense_128"),
    tf.keras.layers.Dropout(0.8, name="Dropout_0.8"),
    tf.keras.layers.Dense(25, activation="sigmoid", name="Output"),
], name="BiLSTM_Model")

# Ensure SEQUENCE_LENGTH is defined
SEQUENCE_LENGTH = 100  # Adjust based on your TextVectorization layer configuration

# Build the model with the correct input shape
lstm_model.build((None, SEQUENCE_LENGTH))

# Print model summary to verify
lstm_model.summary()


Model: "BiLSTM_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ text_vectorization                   │ (None, 100)                 │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 100, 256)            │       2,560,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ BiLSTM_256 (Bidirectional)           │ (None, 100, 512)            │       1,050,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ BiLSTM_128 (Bidirectional)           │ (None, 256)                 │         656,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dense_128 (Dense)                    │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dropout_0.8 (Dropout)                │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Output (Dense)                       │ (None, 25)                  │           3,225 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,303,129 (16.42 MB)

 Trainable params: 4,303,129 (16.42 MB)

 Non-trainable params: 0 (0.00 B)

* Print the model summary and confirm is has the same architecture as the outline provided above.

In [20]:
# CodeGrade Tag Ex2b-iii
### Print the model summary and confirm it has the same architecture as the
### outline provided above.

# Print the LSTM model summary
lstm_model.summary()


Model: "BiLSTM_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ text_vectorization                   │ (None, 100)                 │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 100, 256)            │       2,560,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ BiLSTM_256 (Bidirectional)           │ (None, 100, 512)            │       1,050,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ BiLSTM_128 (Bidirectional)           │ (None, 256)                 │         656,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dense_128 (Dense)                    │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dropout_0.8 (Dropout)                │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Output (Dense)                       │ (None, 25)                  │           3,225 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,303,129 (16.42 MB)

 Trainable params: 4,303,129 (16.42 MB)

 Non-trainable params: 0 (0.00 B)

* Compile the model with binary crossentropy loss and the adam optimizer. For the metrics, use the Precision and Recall functions.

In [21]:
# CodeGrade Tag Ex2b-iv
### Compile the model with binary crossentropy loss, the adam optimizer, with
### the precision and recall metrics

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

lstm_model.compile(
    optimizer=Adam(learning_rate=1e-4),  # Adam optimizer with a learning rate of 1e-4
    loss='binary_crossentropy',         # Binary crossentropy loss
    metrics=[Precision(), Recall()]     # Metrics: Precision and Recall
)


# 3. Training of the models

* **For each model, create a Checkpoint Callback that saves the weights of the best performing epoch, based on the validation loss.**

In [23]:
# CodeGrade Tag Ex3a-i
### Create two ModelCheckpoint callbacks to store the best weights from each
### model, both based on the validation loss.

# Define filepaths for saving weights
checkpoint_cnn_filepath = '/content/checkpoint_cnn.weights.h5'
checkpoint_lstm_filepath = '/content/checkpoint_lstm.weights.h5'

from tensorflow.keras.callbacks import ModelCheckpoint

# Checkpoint for the CNN model
checkpoint_cnn_callback = ModelCheckpoint(
    filepath=checkpoint_cnn_filepath,   # Filepath to save the CNN weights
    save_weights_only=True,             # Save only the model weights
    monitor='val_loss',                 # Monitor validation loss
    mode='min',                         # Minimize validation loss
    save_best_only=True,                # Save only the best weights
    verbose=1                           # Display message when weights are saved
)

# Checkpoint for the LSTM model
checkpoint_lstm_callback = ModelCheckpoint(
    filepath=checkpoint_lstm_filepath,  # Filepath to save the LSTM weights
    save_weights_only=True,             # Save only the model weights
    monitor='val_loss',                 # Monitor validation loss
    mode='min',                         # Minimize validation loss
    save_best_only=True,                # Save only the best weights
    verbose=1                           # Display message when weights are saved
)


* **Create a Learning Rate Scheduler Callback that utilises the provided function to decrease the learning rate during training.**

In [24]:
# CodeGrade Tag Ex3a-ii
### Using the function provided, create a LearningRateScheduler callback, call
### it "lr_callback"

from tensorflow.keras.callbacks import LearningRateScheduler

def scheduler(epoch, lr):
    if epoch < 10:
        return float(lr)
    else:
        return float(lr * tf.math.exp(-0.01))

# Create the LearningRateScheduler callback
lr_callback = LearningRateScheduler(schedule=scheduler, verbose=1)



### 3.a. CNN training

* **Train the CNN model for 40 epochs, using the callbacks you made previously. Store the losses and metrics to use later.**

In [31]:
# CodeGrade Tag Ex3a-iii
### Train the model for 40 epochs, using the callbacks you have created. Store
### the losses and metrics in a history object.

cnn_history = cnn_model.fit(
    posters_train_ds,
    validation_data=posters_valid_ds,
    epochs=40,
    callbacks=[checkpoint_cnn_callback, lr_callback]
    )


Epoch 1: LearningRateScheduler setting learning rate to 7.40818286431022e-05.
Epoch 1/40
79/80 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.2161 - precision: 0.6427 - recall: 0.2939
Epoch 1: val_loss did not improve from 0.25356
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.2162 - precision: 0.6427 - recall: 0.2937 - val_loss: 0.2555 - val_precision: 0.6056 - val_recall: 0.2820 - learning_rate: 7.4082e-05

Epoch 2: LearningRateScheduler setting learning rate to 7.40818286431022e-05.
Epoch 2/40
79/80 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.2163 - precision: 0.6429 - recall: 0.2977
Epoch 2: val_loss improved from 0.25356 to 0.25285, saving model to /content/checkpoint_cnn.weights.h5
80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 55ms/step - loss: 0.2163 - precision: 0.6429 - recall: 0.2974 - val_loss: 0.2529 - val_precision: 0.6066 - val_recall: 0.2817 - learning_rate: 7.4082e-05

Epoch 3: LearningRateScheduler setting learning rate to 7.40818286431022e-05.
Epoch 3/40
79/80 ━━━━━━━━━━━━━━━━━━━━ 0

* **Train the model for 20 epochs** (this may take several minutes)**, using the callbacks you made previously. Store the losses and metrics to use later.**

In [34]:
# Define sequence length consistent with the TextVectorization layer
SEQUENCE_LENGTH = 100

# Preprocess function to tokenize and pad sequences
def preprocess_overview(overview, label):
    # Tokenize and pad sequences to SEQUENCE_LENGTH using the encoder
    overview = encoder(overview)  # Convert raw text to tokenized sequences
    return overview, label

# Apply preprocessing and batch the datasets
processed_overviews_train_ds = (overviews_train_ds
                                .map(preprocess_overview, num_parallel_calls=tf.data.AUTOTUNE)
                                .batch(64)
                                .prefetch(tf.data.AUTOTUNE))

processed_overviews_valid_ds = (overviews_valid_ds
                                .map(preprocess_overview, num_parallel_calls=tf.data.AUTOTUNE)
                                .batch(64)
                                .prefetch(tf.data.AUTOTUNE))

# Train the LSTM model
lstm_history = lstm_model.fit(
    processed_overviews_train_ds,         # Preprocessed training dataset
    validation_data=processed_overviews_valid_ds,  # Preprocessed validation dataset
    epochs=20,                            # Number of epochs
    callbacks=[checkpoint_lstm_callback, lr_callback],  # Callbacks for training
    verbose=1                             # Verbosity level
)



Epoch 1: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 1/20


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("BiLSTM_Model_1/Cast:0", shape=(None, None, None), dtype=string). Expected shape (None, 100), but input has incompatible shape (None, None, None)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, None, None), dtype=int64)
  • training=True
  • mask=None

# 4. Evaluation of the models

### 4.a. CNN Evaluation

* **Create plots using the losses and metrics. In your report, discuss these results and critically evaluate the model performance.**

In [ ]:
# CodeGrade Tag Ex4a-i

#Complete here

* **Load the best weights from your model checkpoint, and create plots demonstrating the classification performance for all three classes. Include these plots in your report, and critically evaluate on the performance of the model across the classes.**

### 4.b. LSTM Evaluation

* **Create plots using the losses and metrics. In your report, discuss these results and critically evaluate the model performance.**

In [ ]:
# CodeGrade Tag Ex4b-i

#Complete here

### 4.c. Produce examples for the report

* First, load the best weights from your checkpoints of both your models.

* Choose a few films from the dataset, plot their posters and print their overviews. Use these example films to demonstrate the classification performance of the CNN model on their posters and of the LSTM model on their overview.

* Be sure to demonstrate the results of the multi-label classification. Compare, for each example film, the top three most probable genres predicted by the CNN and the top three most probable genres predicted by the LSTM with the ground truth genres.

* Include these examples in your report, and critically evaluate on the performance of the model across the classes.

In [ ]:
# CodeGrade Tag Ex4c

#Complete here